# 2. OpenSearch Hands-On (Part1)

## 2.1. OpenSearch との接続

In [1]:
import pprint
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts=[{"host": "localhost", "port": 9200}],
    http_compress=True,  # 圧縮有効化
)

# クラスタ情報の確認
pprint.pprint(client.info())

{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'u_s80VD9RSCBTiz2OAThVg',
 'name': '9e342b1d841b',
 'tagline': 'The OpenSearch Project: https://opensearch.org/',
 'version': {'build_date': '2024-10-31T19:08:04.231254959Z',
             'build_hash': '99a9a81da366173b0c2b963b26ea92e15ef34547',
             'build_snapshot': False,
             'build_type': 'tar',
             'distribution': 'opensearch',
             'lucene_version': '9.12.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.10.0',
             'number': '2.18.0'}}


## 2.2. インデックスの作成

- OpenSearch に sample-index というインデックスを作成 する
- データ構造（mappings）を定義する
- データのスキーマ（データの型の定義） を設定
- 全文検索ができる title と description を持つ
- timestamp を date 型として扱う
- すでに存在する場合はエラーを無視 する (ignore=400)

In [2]:
# インデックス名の定義
index_name = "sample-index"

# インデックス作成
response = client.indices.create(
    index=index_name,
    body={
        "settings": {
            "index": {
                "number_of_shards": 1,   # インデックスを1つのシャードに分割
                "number_of_replicas": 1  # データのバックアップを1つ作成
            }
        },
        "mappings": {
            "properties": {
                "title": {"type": "text"},        # タイトルフィールド（全文検索可能な text 型）
                "description": {"type": "text"},  # 説明フィールド（全文検索可能な text 型）
                "timestamp": {"type": "date"}     # タイムスタンプフィールド（date 型で日時管理）
            }
        }
    },
    ignore=400  # インデックスがすでに存在する場合にエラーを無視
)

# インデックス作成のレスポンスを出力
pprint.pprint(response)

{'acknowledged': True, 'index': 'sample-index', 'shards_acknowledged': True}


## 2.3. データの投入

### 2.3.1 データの投入

In [3]:
import datetime

doc = {
    "title": "OpenSearch Hands-On",
    "description": "Python から OpenSearch を扱うハンズオンです。",
    "timestamp": datetime.datetime.now()
}

# ドキュメントをインデックスに追加
response = client.index(
    index=index_name,
    body=doc,
    id=1,  # ドキュメントIDを指定
    refresh=True
)
pprint.pprint(response)

{'_id': '1',
 '_index': 'sample-index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'forced_refresh': True,
 'result': 'created'}


### 2.3.1. 投入したデータの確認

In [4]:
# 更新後のデータを取得
response = client.get(index=index_name, id=1)

# 更新後の description を表示
pprint.pprint(response)

{'_id': '1',
 '_index': 'sample-index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_source': {'description': 'Python から OpenSearch を扱うハンズオンです。',
             'timestamp': '2025-02-11T19:16:10.739500',
             'title': 'OpenSearch Hands-On'},
 '_version': 1,
 'found': True}


## 2.4. データの検索

In [5]:
query = {
    "query": {
        "match": {
            "title": "OpenSearch"
        }
    }
}

response = client.search(
    index=index_name,
    body=query
)

pprint.pprint(response)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'sample-index',
                    '_score': 0.2876821,
                    '_source': {'description': 'Python から OpenSearch '
                                               'を扱うハンズオンです。',
                                'timestamp': '2025-02-11T19:16:10.739500',
                                'title': 'OpenSearch Hands-On'}}],
          'max_score': 0.2876821,
          'total': {'relation': 'eq', 'value': 1}},
 'timed_out': False,
 'took': 15}


## 2.5. データの更新

### 2.5.1. docの更新

In [6]:
update_doc = {
    "doc": {
        "description": "OpenSearch の基本操作を学ぶハンズオン"
    }
}

response = client.update(
    index=index_name,
    id=1,
    body=update_doc
)
pprint.pprint(response)

{'_id': '1',
 '_index': 'sample-index',
 '_primary_term': 1,
 '_seq_no': 1,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 2,
 'result': 'updated'}


### 2.5.1. 更新した内容の確認

In [7]:
# 更新後のデータを取得
response = client.get(index=index_name, id=1)

# 更新後の description を表示
print(response["_source"]["description"])

OpenSearch の基本操作を学ぶハンズオン


## 2.6. データの削除

In [8]:
response = client.delete(
    index=index_name,
    id=1
)
pprint.pprint(response)

{'_id': '1',
 '_index': 'sample-index',
 '_primary_term': 1,
 '_seq_no': 2,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 3,
 'result': 'deleted'}


## 2.7. インデックスの削除

In [9]:
# 作成したインデックスを削除
response = client.indices.delete(
    index=index_name
)

# インデックス削除の結果を出力
pprint.pprint(response)

{'acknowledged': True}
